In [1]:
import requests as rq
import json
import pandas as pd
import time
from dotomorrow import SavedIterator

In [2]:
def timestamp_onth_start_end(year, month):
    if month < 12:
        start_date = f"{year}-{month:02}-01T00:00:00"
        end_date = f"{year}-{month+1:02}-01T00:00:00"
    else:
        start_date = f"{year}-{month:02}-01T00:00:00"
        end_date = f"{year+1}-01-01T00:00:00"
    return start_date, end_date

In [22]:
timestamp_onth_start_end(2021, 1)

('2021-01-01T00:00:00', '2021-02-01T00:00:00')

In [3]:
# STATION_IDs = 'C0A980'
# STATION_TYPE= 'auto_C0'
# TYPE = 'report_date'

# s,e = timestamp_onth_start_end(2021,1)

# req = rq.post(
#     'https://codis.cwa.gov.tw/api/station',
#     {
#         'type': TYPE,
#         'stn_ID': STATION_IDs,
#         'stn_type': STATION_TYPE,
#         'start':s,
#         'end':e
#     }
# )

In [3]:
def load_from_rq(req):
    r = req.json()
    if r['code'] != 200:
        raise ValueError("Request Fail!!")
    datas = r['data'][0]['dts']
    station_id = r['data'][0]['StationID']
    results = []
    for data in datas:
        results.append(
            dict(
        datetime = data['DataTime'],
        temperature = data['AirTemperature']['Instantaneous'],
        relative_humidity = data['RelativeHumidity']['Instantaneous'],
        wind_speed = data['WindSpeed']['Mean'],
        precipitation = data['Precipitation']['Accumulation'],
        )
    )
    df = pd.DataFrame(results)
    df['station_id'] = station_id

    return df



In [4]:
station_list = pd.read_csv("./DATA/station_list.csv")
station_list_test = station_list[1:2]
station_list

,station_name,station_id,station_type,latitude,longitude
0,社子,C0A980,auto_C0,25.1095,121.4697
1,天母,C0A9C0,auto_C0,25.1175,121.5372
2,內湖,C0A9F0,auto_C0,25.0794,121.5755
3,大屯山,C0AC40,auto_C0,25.1757,121.5224
4,信義,C0AC70,auto_C0,25.0378,121.5646
5,文山,C0AC80,auto_C0,25.0023,121.5757
6,平等,C0AH40,auto_C0,25.1291,121.5771
7,松山,C0AH70,auto_C0,25.0487,121.5504
8,石牌,C0AI40,auto_C0,25.1156,121.5131
9,鞍部,466910,cwb,25.1826,121.5297


In [ ]:
def get_data_for_all_stations():
    import random

    all_dataframes = pd.DataFrame(
        columns = ['datetime', 'temperature', 'relative_humidity', 'wind_speed', 'precipitation', 'station_id'])

    for index, row in station_list_test.iterrows():
        STATION_ID = row['station_id']
        STATION_TYPE = row['station_type']
        TYPE = 'report_date'

        for year in range(2021, 2022):
            for month in range(1, 6):
                s, e = timestamp_onth_start_end(year, month)

                # user_agent = random.choice(user_agents)
                # headers = {'User-Agent': user_agent}

                req = rq.post(
                    'https://codis.cwa.gov.tw/api/station',
                    {
                        'type': TYPE,
                        'stn_ID': STATION_ID,
                        'stn_type': STATION_TYPE,
                        'start': s,
                        'end': e
                    }
                )

                sleep_duration = random.uniform(10, 20)
                time.sleep(sleep_duration)  # wait for 2 seconds before the next request

                df = load_from_rq(req)
                all_dataframes = pd.concat([all_dataframes, df], axis=0, ignore_index=True)
                print(f"STATION_ID {STATION_ID}, {year}/{month} processed")

    return all_dataframes

# Get data for all stations and time periods
all_station_data = get_data_for_all_stations()

# Save the data to a CSV file or do further processing as needed
# all_station_data.to_csv("all_station_data.csv", index=False)